# Gallery Example: M/Hyper/k Queue

This example demonstrates an M/Hyper/k queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Coxian distribution fitted to high SCV
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

The high variability service with multiple servers shows how added capacity can mitigate service variability.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mhypk(k=2): Network {
    """Create M/Hyper/k queueing model"""
    val model = Network(f"M/Hyper/{k}")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
    source.setArrival(oclass, Exp(1))
// Coxian service with mean=0.5 and SCV=4 (high variability)
    queue.setService(oclass, Coxian.fit_mean_and_scv(0.5, 4))
    queue.setNumberOfServers(k)
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model with k=2 servers
val model = gallery_mhypk(k=2)

## About M/Hyper/k Queues

For M/Hyper/k with k=2 servers:
- **Arrival rate**: λ = 1
- **Service mean**: 0.5 (rate μ = 2 per server)
- **Service SCV**: 4 (high variability, Coxian)
- **Total capacity**: k × μ = 4
- **Utilization**: ρ = λ / (k × μ) = 1/4 = 0.25

The high service variability (SCV = 4) means:
- Some jobs get very fast service
- Others require much longer
- Multiple servers help smooth out the variability

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)